In [1]:
import pandas as pd
from pymongo import MongoClient

# Load the CSV data
file_path = '/Users/nicholasnoto/Desktop/CIS 4400/Homework1/Mental_Health_Care_in_the_Last_4_Weeks.csv'
df = pd.read_csv(file_path)

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['mental_health']

# Create and populate dimension tables
def create_dimension_table(df, column_name):
    unique_values = df[[column_name]].drop_duplicates().reset_index(drop=True)
    unique_values = unique_values.rename(columns={column_name: 'value'})
    unique_values['_id'] = unique_values.index + 1
    collection_name = f'dim_{column_name.lower()}'
    db[collection_name].insert_many(unique_values.to_dict('records'))
    return db[collection_name]

# Creating dimension tables
dim_group = create_dimension_table(df, 'Group')
dim_state = create_dimension_table(df, 'State')
dim_subgroup = create_dimension_table(df, 'Subgroup')
dim_phase = create_dimension_table(df, 'Phase')
dim_time_period = create_dimension_table(df, 'Time Period')
dim_time_period_label = create_dimension_table(df, 'Time Period Label')

# Replace dimension values in fact table with their respective IDs
def replace_with_id(df, column_name):
    collection_name = f'dim_{column_name.lower()}'
    dim_collection = db[collection_name]
    dim_map = {doc['value']: doc['_id'] for doc in dim_collection.find({})}
    df[column_name] = df[column_name].map(dim_map)
    return df

# Replace values with IDs in the fact table
fact_table = df.copy()
fact_table = replace_with_id(fact_table, 'Group')
fact_table = replace_with_id(fact_table, 'State')
fact_table = replace_with_id(fact_table, 'Subgroup')
fact_table = replace_with_id(fact_table, 'Phase')
fact_table = replace_with_id(fact_table, 'Time Period')
fact_table = replace_with_id(fact_table, 'Time Period Label')

# Create and populate the fact table
fact_table_name = 'fact_mental_health_care'
db[fact_table_name].insert_many(fact_table.to_dict('records'))

print("Fact and dimension tables created and populated successfully")


Fact and dimension tables created and populated successfully
